# YOLO Fine-Tuning su LineMOD

Versione minimale e pulita del processo di fine-tuning YOLO su LineMOD:
1. Caricamento modello YOLO pre-addestrato
2. Preparazione dataset
3. Fine-tuning della head
4. Valutazione su test set


## 1. Import e setup

In [ ]:
# Abilita visualizzazione inline di matplotlib
%matplotlib inline

# 1. Import e setup
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import random
from PIL import Image
import numpy as np

sys.path.insert(0, str(Path.cwd().parent))
from config import Config
from models.yolo_detector import YOLODetector
from utils.prepare_yolo_symlinks import prepare_yolo_dataset_symlinks, create_data_yaml
from utils.organize_yolo_results import organize_yolo_output, print_organization_summary

## 2. Carica modello YOLO e prepara dataset

In [ ]:
# 2. Carica modello YOLO e prepara dataset

detector = YOLODetector(
    model_name=Config.YOLO_MODEL,  # usa modello pre-addestrato
    pretrained=True,
    num_classes=Config.NUM_CLASSES
)

# Prepara dataset (symlinks e data.yaml)
output_dir = Config.LINEMOD_ROOT / 'yolo_symlinks'
if (output_dir / 'data.yaml').exists():
    data_yaml_path = output_dir / 'data.yaml'
else:
    prepare_yolo_dataset_symlinks(
        dataset_root=Config.LINEMOD_ROOT,
        output_root=output_dir,
        use_symlinks=True
    )
    data_yaml_path = create_data_yaml(output_dir, Config.DATASETS_DIR)
print(f"Dataset pronto: {data_yaml_path}")

### Verifica Dataset

Prima di procedere, verifichiamo che il dataset sia stato preparato correttamente.

In [ ]:
# Verifica che il dataset sia stato creato correttamente
train_images = list((output_dir / 'images' / 'train').glob('*.png'))
val_images = list((output_dir / 'images' / 'val').glob('*.png'))
train_labels = list((output_dir / 'labels' / 'train').glob('*.txt'))
val_labels = list((output_dir / 'labels' / 'val').glob('*.txt'))

print(f"✅ Dataset YOLO preparato:")
print(f"   Train: {len(train_images)} immagini, {len(train_labels)} labels")
print(f"   Val:   {len(val_images)} immagini, {len(val_labels)} labels")
print(f"   Classi: {Config.NUM_CLASSES} ({', '.join(Config.CLASS_NAMES)})")
print(f"   Config: {data_yaml_path}")

## 3. Fine-tuning solo della head

In [ ]:
# 3. Fine-tuning solo della head
# Definizione path
training_output_dir = Config.CHECKPOINT_DIR / 'yolo' / f'yolo_train{Config.YOLO_EPOCHS}'
yolo_temp_dir = Path('runs/detect/val')  # directory temporanea dove YOLO potrebbe salvare file

# Congela backbone, allena solo la head
detector.freeze_backbone(freeze_until_layer=Config.YOLO_FREEZE_UNTIL_LAYER)

results = detector.train(
    data_yaml=str(data_yaml_path),
    epochs=Config.YOLO_EPOCHS,
    imgsz=Config.YOLO_IMG_SIZE,
    batch_size=Config.YOLO_BATCH_SIZE,
    project=str(Config.CHECKPOINT_DIR / 'yolo'),
    name=f'yolo_train{Config.YOLO_EPOCHS}',
    save=True,
    cache=True,
    amp=Config.AMP_YOLO,
    workers=Config.NUM_WORKERS,
    verbose=True
)
print("Training completato.")

# Organizza automaticamente i risultati in sottocartelle
# Se YOLO ha creato file temporanei in runs/detect/val, li spostiamo
if yolo_temp_dir.exists() and any(yolo_temp_dir.iterdir()):
    print(f"🔄 Organizzazione file da {yolo_temp_dir} a {training_output_dir}")
    stats = organize_yolo_output(run_dir=yolo_temp_dir, destination_dir=training_output_dir)
    print_organization_summary(training_output_dir, stats)
else:
    print(f"ℹ️  Nessun file temporaneo da organizzare in {yolo_temp_dir}")
    # Organizza i file direttamente nella directory di output se esistono
    if training_output_dir.exists():
        print(f"📁 File di training salvati in: {training_output_dir}")

## 4. Valutazione su test set

In [ ]:
# 4. Valutazione su test set
YOLO_NAME = f'yolo_train{Config.YOLO_EPOCHS}'
weights_path = Config.CHECKPOINT_DIR / 'yolo' / YOLO_NAME / 'weights' / 'best.pt'

if weights_path.exists():
    finetuned_detector = YOLODetector(model_name=str(weights_path), num_classes=Config.NUM_CLASSES)
    metrics = finetuned_detector.validate(
        data_yaml=str(data_yaml_path),
        conf=Config.YOLO_CONF_THRESHOLD,
        verbose=False
    )
    print(f"mAP@0.5: {metrics.box.map50:.4f}")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall: {metrics.box.mr:.4f}")
else:
    print(f"Modello non trovato: {weights_path}")

## 5. Visualizzazione: predizione vs ground truth su immagine casuale

In [ ]:
# 5. Visualizzazione: predizione vs ground truth su immagine casuale
images_dir = Config.LINEMOD_ROOT / 'yolo_symlinks' / 'images' / 'val'
labels_dir = Config.LINEMOD_ROOT / 'yolo_symlinks' / 'labels' / 'val'

image_files = sorted([p for p in images_dir.iterdir() if p.is_file()])
img_path = random.choice(image_files)
label_path = labels_dir / (img_path.stem + '.txt')

# Carica immagine
pil_img_RGB = np.array(Image.open(img_path).convert('RGB'))
pil_img_BGR = pil_img_RGB[..., ::-1]  # YOLO si aspetta BGR (formato OpenCV)
H, W = pil_img_RGB.shape[:2]

# Predizione YOLO
if 'finetuned_detector' not in globals():
    finetuned_detector = YOLODetector(model_name=str(weights_path), num_classes=Config.NUM_CLASSES)
detections = finetuned_detector.detect_objects(pil_img_BGR, conf_threshold=0.2)

# Visualizza GT (rosso) e predizioni (verde)
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(pil_img_RGB)

# GT: file label YOLO-format: class x_center y_center w h (normalized)
if label_path.exists():
    with open(label_path) as f:
        for line in f:
            if not line.strip():
                continue
            cls, xc, yc, wn, hn = line.split()
            xc, yc, wn, hn = map(float, (xc, yc, wn, hn))
            x0 = (xc - wn/2) * W
            y0 = (yc - hn/2) * H
            x1 = (xc + wn/2) * W
            y1 = (yc + hn/2) * H
            rect = plt.Rectangle((x0, y0), x1-x0, y1-y0, edgecolor='red', facecolor='none', linewidth=2)
            ax.add_patch(rect)
            ax.text(x0, y0-8, f'GT {int(cls)}', color='red', fontsize=11, backgroundcolor='white')
else:
    print('GT label file non trovato:', label_path)

# Predizioni YOLO (verde)
for det in detections:
    if 'xyxy' in det:
        x0, y0, x1, y1 = det['xyxy']
    elif 'bbox' in det:
        x0, y0, x1, y1 = det['bbox']
    else:
        continue
    name = det.get('class_name', det.get('class', '?'))
    conf = det.get('confidence', det.get('score', 0))
    rect = plt.Rectangle((x0, y0), x1-x0, y1-y0, edgecolor='green', facecolor='none', linewidth=2)
    ax.add_patch(rect)
    ax.text(x0, y1+6, f'{name} {conf:.2f}', color='green', fontsize=11, backgroundcolor='white')

ax.set_axis_off()
plt.title('GT (rosso) vs YOLO prediction (verde)')
plt.tight_layout()
plt.show()